## User Management
Interaction with EOEPCA endpoint using python Demo Client.

In [16]:
import utils.DemoKeycloakClient as client
import jwt
import json

## Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorisation is obtained for scoped access resources on behalf of the user.

In [17]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
# TODO: change base_domain to demo.eoepca.org once deployed
base_domain = "develop.eoepca.org"
platform_domain = f"keycloak.{base_domain}"
server_url = f"https://{platform_domain}"
realm = "master"
authorize_url = f"{server_url}/realms/master/protocol/openid-connect/auth"
token_url = f"{server_url}/realms/master/protocol/openid-connect/token"
open_service = "https://dummy-service-open.develop.eoepca.org"
protected_service = "https://dummy-service.develop.eoepca.org"

demo = client.DemoKeycloakClient(server_url, realm, "admin", "admin_Abcd1234#")

> Created client:
{'id': 'c9af850a-e4e4-4108-a6b4-b5e688da0bd0', 'clientId': 'demo', 'surrogateAuthRequired': False, 'enabled': True, 'alwaysDisplayInConsole': False, 'clientAuthenticatorType': 'client-secret', 'secret': '3DRJyrtksRYfpUvZpOHPgQS5f8MIaE5T', 'redirectUris': [], 'webOrigins': [], 'notBefore': 0, 'bearerOnly': False, 'consentRequired': False, 'standardFlowEnabled': True, 'implicitFlowEnabled': False, 'directAccessGrantsEnabled': True, 'serviceAccountsEnabled': True, 'publicClient': False, 'frontchannelLogout': False, 'protocol': 'openid-connect', 'attributes': {'client.secret.creation.time': '1684150587'}, 'authenticationFlowBindingOverrides': {}, 'fullScopeAllowed': True, 'nodeReRegistrationTimeout': -1, 'protocolMappers': [{'id': '93e3804e-fcb2-4d47-a606-b03f868f146a', 'name': 'Client Host', 'protocol': 'openid-connect', 'protocolMapper': 'oidc-usersessionmodel-note-mapper', 'consentRequired': False, 'config': {'user.session.note': 'clientHost', 'id.token.claim': 'true',

## User Authentication
User authenticates and the client receives an ID Token (JWT) that represents the user, and is used to identify the user in UMA authorization flows.

### Create Eric User

In [18]:
USER_NAME="eric"
USER_PASSWORD="defaultPWD"
demo.create_user(USER_NAME, USER_PASSWORD)

Created user: 78519065-1da9-4105-b2d1-4e665079d2bf


### Authenticate

In [27]:
#-------------------------------------------------------------------------------
# Authenticate as Eric and get User Token
#-------------------------------------------------------------------------------
token = demo.get_user_token(USER_NAME, USER_PASSWORD)
print(USER_NAME + " token:\n" + json.dumps(token, indent = 2))

eric token:
{
  "access_token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJXZWFIY2pscThPc1RUYjdlV0s5SjJTTDFBUDIyazZpajdlMGFlVHRNU2xRIn0.eyJleHAiOjE2ODQxNTI2ODAsImlhdCI6MTY4NDE1MjYyMCwianRpIjoiNWIzMWRjMjctMThiYi00NDNhLTllZjgtYTdmMWM2YzRiNjRmIiwiaXNzIjoiaHR0cHM6Ly9rZXljbG9hay5kZXZlbG9wLmVvZXBjYS5vcmcvcmVhbG1zL21hc3RlciIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI3ODUxOTA2NS0xZGE5LTQxMDUtYjJkMS00ZTY2NTA3OWQyYmYiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJkZW1vIiwic2Vzc2lvbl9zdGF0ZSI6ImJhMjk2NTNmLWI4NzktNDFhZC1iMDk3LWQ5MTJiZTQ2NTZmMiIsImFjciI6IjEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1tYXN0ZXIiLCJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJlbWFpbCBwcm9maWxlIiwic2lkIjoiYmEyOTY1M2YtYjg3OS00MWFkLWIwOTctZDkxMmJlNDY1NmYyIiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJlcmljIn0.FNjqYb2tC5f3aYd5ebgAYvm_1Ec-HPestdgsp-GbD30Dy6f0wWWFt_1V5Wl

### Inspect the User Token

In [26]:
# Inspect User Token
access_token = token['access_token']
jwt_header = jwt.get_unverified_header(access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))
uid = jwt_payload["sub"]

JWT Header:
{
  "alg": "RS256",
  "typ": "JWT",
  "kid": "WeaHcjlq8OsTTb7eWK9J2SL1AP22k6ij7e0aeTtMSlQ"
}
JWT Payload:
{
  "exp": 1684152463,
  "iat": 1684152403,
  "jti": "4459350d-c976-486e-8d71-155fc403cc8f",
  "iss": "https://keycloak.develop.eoepca.org/realms/master",
  "aud": "account",
  "sub": "78519065-1da9-4105-b2d1-4e665079d2bf",
  "typ": "Bearer",
  "azp": "demo",
  "session_state": "c276d13b-c376-42fb-a1ec-6ec8f23aeff9",
  "acr": "1",
  "realm_access": {
    "roles": [
      "default-roles-master",
      "offline_access",
      "uma_authorization"
    ]
  },
  "resource_access": {
    "account": {
      "roles": [
        "manage-account",
        "manage-account-links",
        "view-profile"
      ]
    }
  },
  "scope": "email profile",
  "sid": "c276d13b-c376-42fb-a1ec-6ec8f23aeff9",
  "email_verified": false,
  "preferred_username": "eric"
}


The ID Token (JWT) identifies the user via user_name / sub (Subject) fields, and the client via the aud (Audience) field. The JWT is signed and can be verified, using the kid (Key ID) field, via the JWKS endpoint of the Authorization Server.